In [3]:
# -*- coding: utf-8 -*-
"""
Full Pipeline: Deep feature extractors + classical classifiers
with 10-fold cross-validation and soft voting ensemble
"""

# ---------------------------
# 0. Imports
# ---------------------------
import pandas as pd
import numpy as np
import re
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, LSTM, GRU, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')

# ---------------------------
# 1. Reproducibility
# ---------------------------
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ---------------------------
# 2. Load Data
# ---------------------------
train_path = 'adjectives_train.csv'
dev_path   = 'adjectives_dev.csv'
test_path  = 'adjectives_test.csv'

train_df = pd.read_csv(train_path)
dev_df   = pd.read_csv(dev_path)
test_df  = pd.read_csv(test_path)

# Combine train + dev
train_df = pd.concat([train_df, dev_df], ignore_index=True)

text_column  = "review"
label_column = "sentiment_label"

X_train_raw = train_df[text_column]
y_train_raw = train_df[label_column]
X_test_raw  = test_df[text_column]
y_test_raw  = test_df[label_column]

# Label encoding
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train_raw)
y_test  = label_encoder.transform(y_test_raw)
NUM_CLASSES = len(label_encoder.classes_)

# ---------------------------
# 3. Preprocessing
# ---------------------------
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r"[^\w\s]", "", text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return " ".join(tokens)

X_train_processed = X_train_raw.apply(preprocess_text).tolist()
X_test_processed  = X_test_raw.apply(preprocess_text).tolist()

# ---------------------------
# 4. Tokenization
# ---------------------------
MAX_NUM_WORDS = 30000
MAX_SEQ_LEN   = 200
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_processed)
X_train_seq = tokenizer.texts_to_sequences(X_train_processed)
X_test_seq  = tokenizer.texts_to_sequences(X_test_processed)
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')
X_test_pad  = pad_sequences(X_test_seq,  maxlen=MAX_SEQ_LEN, padding='post', truncating='post')

# ---------------------------
# 5. Deep feature extractors (single-layer)
# ---------------------------
def build_cnn(max_seq_len, vocab_size, embedding_dim, feature_dim=128):
    inp = Input(shape=(max_seq_len,))
    x = Embedding(vocab_size, embedding_dim)(inp)
    x = Conv1D(128, 5, activation='relu', padding='same')(x)
    x = GlobalMaxPooling1D()(x)
    feat = Dense(feature_dim, activation='relu', name='feat')(x)
    out = Dense(NUM_CLASSES, activation='softmax')(feat)
    model = Model(inputs=inp, outputs=out)
    feat_extractor = Model(inputs=inp, outputs=feat)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(1e-3), metrics=['accuracy'])
    return model, feat_extractor

def build_lstm(max_seq_len, vocab_size, embedding_dim, feature_dim=128):
    inp = Input(shape=(max_seq_len,))
    x = Embedding(vocab_size, embedding_dim)(inp)
    x = LSTM(128)(x)
    feat = Dense(feature_dim, activation='relu', name='feat')(x)
    out = Dense(NUM_CLASSES, activation='softmax')(feat)
    model = Model(inputs=inp, outputs=out)
    feat_extractor = Model(inputs=inp, outputs=feat)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(1e-3), metrics=['accuracy'])
    return model, feat_extractor

def build_bilstm(max_seq_len, vocab_size, embedding_dim, feature_dim=128):
    inp = Input(shape=(max_seq_len,))
    x = Embedding(vocab_size, embedding_dim)(inp)
    x = Bidirectional(LSTM(128))(x)
    feat = Dense(feature_dim, activation='relu', name='feat')(x)
    out = Dense(NUM_CLASSES, activation='softmax')(feat)
    model = Model(inputs=inp, outputs=out)
    feat_extractor = Model(inputs=inp, outputs=feat)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(1e-3), metrics=['accuracy'])
    return model, feat_extractor

def build_gru(max_seq_len, vocab_size, embedding_dim, feature_dim=128):
    inp = Input(shape=(max_seq_len,))
    x = Embedding(vocab_size, embedding_dim)(inp)
    x = GRU(128)(x)
    feat = Dense(feature_dim, activation='relu', name='feat')(x)
    out = Dense(NUM_CLASSES, activation='softmax')(feat)
    model = Model(inputs=inp, outputs=out)
    feat_extractor = Model(inputs=inp, outputs=feat)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(1e-3), metrics=['accuracy'])
    return model, feat_extractor

def build_cnn_gru(max_seq_len, vocab_size, embedding_dim, feature_dim=128):
    inp = Input(shape=(max_seq_len,))
    x = Embedding(vocab_size, embedding_dim)(inp)
    x = Conv1D(128, 5, activation='relu', padding='same')(x)
    x = GRU(128)(x)
    feat = Dense(feature_dim, activation='relu', name='feat')(x)
    out = Dense(NUM_CLASSES, activation='softmax')(feat)
    model = Model(inputs=inp, outputs=out)
    feat_extractor = Model(inputs=inp, outputs=feat)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(1e-3), metrics=['accuracy'])
    return model, feat_extractor

def build_cnn_lstm(max_seq_len, vocab_size, embedding_dim, feature_dim=128):
    inp = Input(shape=(max_seq_len,))
    x = Embedding(vocab_size, embedding_dim)(inp)
    x = Conv1D(128, 5, activation='relu', padding='same')(x)
    x = LSTM(128)(x)
    feat = Dense(feature_dim, activation='relu', name='feat')(x)
    out = Dense(NUM_CLASSES, activation='softmax')(feat)
    model = Model(inputs=inp, outputs=out)
    feat_extractor = Model(inputs=inp, outputs=feat)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(1e-3), metrics=['accuracy'])
    return model, feat_extractor

VOCAB_SIZE = min(MAX_NUM_WORDS, len(tokenizer.word_index)+1)
BACKBONES = {
    "CNN": build_cnn,
    "LSTM": build_lstm,
    "BiLSTM": build_bilstm,
    "GRU": build_gru,
    "CNN-GRU": build_cnn_gru,
    "CNN-LSTM": build_cnn_lstm
}

# ---------------------------
# 6. Classical classifiers
# ---------------------------
def build_classifiers():
    clfs = [
        ("nb", MultinomialNB(alpha=1.0)),
        ("svm", LinearSVC(C=1.0, dual=False, max_iter=5000, random_state=RANDOM_SEED)),
        ("rf", RandomForestClassifier(n_estimators=200, max_depth=None, random_state=RANDOM_SEED)),
        ("xgb", XGBClassifier(use_label_encoder=False, eval_metric="mlogloss", n_estimators=100, max_depth=3, learning_rate=0.1, random_state=RANDOM_SEED))
    ]
    return clfs

# ---------------------------
# 7. 10-Fold CV + Soft Voting
# ---------------------------
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_SEED)
fold_results = []

for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_train_pad, y_train)):
    X_tr, X_val = X_train_pad[train_idx], X_train_pad[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]
    y_tr_cat, y_val_cat = to_categorical(y_tr, num_classes=NUM_CLASSES), to_categorical(y_val, num_classes=NUM_CLASSES)

    # --- Deep feature extraction ---
    train_features, val_features = [], []
    for name, builder in BACKBONES.items():
        print(f"Training {name} backbone on fold {fold_idx+1}...")
        model, feat_extractor = builder(MAX_SEQ_LEN, VOCAB_SIZE, EMBEDDING_DIM)
        es = EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True, verbose=0)
        model.fit(X_tr, y_tr_cat, validation_split=0.1, epochs=6, batch_size=64, callbacks=[es], verbose=0)
        feat_tr = feat_extractor.predict(X_tr, batch_size=64, verbose=0)
        feat_val = feat_extractor.predict(X_val, batch_size=64, verbose=0)
        train_features.append(feat_tr)
        val_features.append(feat_val)
        tf.keras.backend.clear_session()

    X_tr_feat = np.concatenate(train_features, axis=1)
    X_val_feat = np.concatenate(val_features, axis=1)

    # --- Classical classifiers ---
    clfs = build_classifiers()
    voting_clf = VotingClassifier(estimators=clfs, voting='soft')
    voting_clf.fit(X_tr_feat, y_tr)
    y_val_pred = voting_clf.predict(X_val_feat)

    # --- Evaluate fold ---
    acc = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, average="weighted", zero_division=0)
    recall = recall_score(y_val, y_val_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_val, y_val_pred, average="weighted", zero_division=0)
    print(f"Fold {fold_idx+1} - Acc: {acc:.4f}, Prec: {precision:.4f}, Rec: {recall:.4f}, F1: {f1:.4f}")

    fold_results.append({
        "Fold": fold_idx + 1,
        "Accuracy": acc,
        "Precision": precision,
        "Recall": recall,
        "F1-score": f1
    })

# ---------------------------
# 8. Summary
# ---------------------------
df_results = pd.DataFrame(fold_results)
print("\n10-Fold CV Results (Soft Voting Ensemble):")
print(df_results)
print("\nMean Scores:")
print(df_results.mean())



========== CNN | Fold 1/10 ==========
Epoch   1/100 - acc: 77.19% - prec: 74.66% - rec: 73.91% - f1: 72.82%
Epoch   2/100 - acc: 78.53% - prec: 76.19% - rec: 75.14% - f1: 73.63%
Epoch   3/100 - acc: 79.19% - prec: 77.38% - rec: 75.87% - f1: 75.21%
Epoch   4/100 - acc: 79.92% - prec: 78.43% - rec: 77.17% - f1: 76.46%
Epoch   5/100 - acc: 81.40% - prec: 79.26% - rec: 77.92% - f1: 77.97%
Epoch   6/100 - acc: 82.30% - prec: 80.32% - rec: 79.27% - f1: 78.85%
Epoch   7/100 - acc: 83.47% - prec: 81.28% - rec: 79.58% - f1: 79.51%
Epoch   8/100 - acc: 83.65% - prec: 82.45% - rec: 80.68% - f1: 80.52%
Epoch   9/100 - acc: 84.69% - prec: 83.21% - rec: 81.55% - f1: 81.80%
Epoch  10/100 - acc: 85.36% - prec: 83.87% - rec: 82.01% - f1: 82.53%
--- Fold 1 Final ---
Accuracy: 85.36% | Precision: 83.87% | Recall: 82.01% | F1: 82.53%

========== CNN | Fold 2/10 ==========
Epoch  11/100 - acc: 86.23% - prec: 85.02% - rec: 82.63% - f1: 83.19%
Epoch  12/100 - acc: 86.84% - prec: 85.54% - rec: 83.15% - f1: 8